In [1]:
from rdflib import Graph
from pydantic import BaseModel, validator, Field
from typing import Optional, Union
from pyfuseki import FusekiUpdate
import json
from datetime import date, datetime
from api.src.function.bibframe.Work.graph import MakeGraph
import httpx
from api.src.schemas.bibframe.work import Work
import pysolr  

In [19]:
g = Graph()
g.parse('http://id.loc.gov/resources/works/21291853.nt', format='nt')
g.serialize('loc1.ttl', format='ttl')

<Graph identifier=Na57cfb0002414db99f53ba7608959db3 (<class 'rdflib.graph.Graph'>)>

In [10]:
g = Graph()
g.parse('preview.rdf')
g.serialize('preview.ttl', format='ttl')

<Graph identifier=Nfb0e782c07634581b030626aa4e4a2e0 (<class 'rdflib.graph.Graph'>)>

In [2]:
with open('api/src/config/authorized_values.json') as file:
    reader = file.read()
    vocabulary = json.loads(reader)
    file.close()

In [39]:
with open('work.json', encoding='utf-8') as file:
    reader = file.read()
    work = json.loads(reader)
    file.close()
request = Work(**work)

In [3]:
request.adminMetadata.dict()

{'encodingLevel': 'menclvl:1',
 'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
 'creationDate': '2023-04-20',
 'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
 'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
 'generationProcess': {'label': 'BiblioKeia',
  'generationDate': '2023-04-20T10:06:11'},
 'status': {'value': 'mstatus:new', 'label': 'novo'}}

In [4]:
request.adminMetadata.status

Status(value='mstatus:new', label='novo')

In [5]:
request = Work(**work)
graph = MakeGraph(request, 'bk-1')
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
        
    INSERT DATA {
        GRAPH bkw:bk-1
        {
                bkw:bk-1 a bf:Monograph, bf:Text, bf:Work  ;
                bf:adminMetadata [ a bf:Ad

In [4]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [11]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = fuseki_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [53]:
g = Graph()
g.parse('https://www.loc.gov/standards/mads/rdf/v1/examples/oca08651008.nt', format='nt')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

In [54]:
g.serialize('EX_NAME.ttl', format='ttl')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

GET WORK

In [45]:
response = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/name/n80002329')
response

<Response [200 OK]>

In [46]:
g = Graph()
g.parse(response.content)
g.serialize('name.ttl', format='ttl')

<Graph identifier=Nc7fd2ec4e1e541edbd048c09e685ea53 (<class 'rdflib.graph.Graph'>)>

In [8]:
g.serialize('bk.rdf', format='xml')

<Graph identifier=Na2c213dac7fc4260b1e9a3ce33f14215 (<class 'rdflib.graph.Graph'>)>

In [51]:
for contribution in request.contribution:
    print(contribution.agent)

https://bibliokeia.com/authorities/name/n80002329


In [42]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = """PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH <{subject}> {{ 
                        <{subject}>  bf:contributionOf   <https://bibliokeia.com/resources/work/{id}> }} }} ; """
for i in request.subject:
    up = subjectOf.format(subject=i.value, id="bk-1")
    response = fuseki_authorities.run_sparql(up)
    print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [32]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = f"""PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH graph:sh85017405 {{ 
                        graph:sh85017405  bf:subjectOf <https://bibliokeia.com/resources/work/bk-2> }} }} ; """
fuseki_authorities.run_sparql(subjectOf)

SOLR

In [40]:
request.genreForm

[GenreForm(label='Ficção', value='genreForms:gf2014026339')]

In [41]:
id = 'bk-1'
doc = {
    'id': id,
    'type': [i.replace("bf:", "") for i in request.type],
    'creationDate': request.adminMetadata.creationDate,
    'status': request.adminMetadata.status.label,
    'cdd': request.classification.classificationPortion,
    'cutter': request.classification.itemPortion,
    'content': [content.label for content in request.content],
    'contribution': [{'id':f'{id}!{contribution.agent.split("/")[-1]}', **contribution.dict()} for contribution in request.contribution],
    'genreForm': [genreForm.label for genreForm in request.genreForm ]
}
doc

{'id': 'bk-1',
 'type': ['Monograph', 'Text', 'Work'],
 'creationDate': '2023-04-20',
 'status': 'novo',
 'cdd': '869.3',
 'cutter': 'M513',
 'content': ['texto', 'Imagem'],
 'contribution': [{'id': 'bk-1!n80002329',
   'type': ['bf:Contribution', 'bflc:PrimaryContribution'],
   'agent': 'https://bibliokeia.com/authorities/name/n80002329',
   'label': 'Machado de Assis',
   'role': 'relators:aut'}],
 'genreForm': ['Ficção']}

In [42]:
solr = pysolr.Solr('http://localhost:8983/solr/acervo/', timeout=10)
r = solr.add([doc], commit=True)
r

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":128}}\n'

In [32]:
solr.delete(q="*:*", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">44</int>\n</lst>\n</response>\n'